In [2]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [3]:
(train_input, train_target) , (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [4]:
# CNN 사용하기 위해서 층이 하나 더 필요하다. 맨 뒤에 오는 그거가 color의 RGB층이 들어가야 된다. 없을 시 그냥 1
# 채널 차원이 더 필요하다.
# why? keras 는 들어오는 데이터가 3차원이라고 가정하고 만든 것이기 때문이다.
train_scaled = train_input.reshape(-1, 28 , 28 , 1) / 255

In [5]:
train_scaled , val_scaled , train_target , val_target = \
    train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

In [6]:
print(train_scaled.shape,train_target.shape)

(48000, 28, 28, 1) (48000,)


In [7]:
print(val_scaled.shape,val_target.shape)

(12000, 28, 28, 1) (12000,)


### 합성곱 층 만들기

In [8]:
model = keras.Sequential()

# 합성곱 층 만들기 : 필터 , 커널 , 패딩 줘서
model.add(keras.layers.Conv2D(
    32, # 필터 개수
    kernel_size = 3, # kernel_size는 무조건 정사각형이다. 값 하나만 주면 됨
    activation='relu',
    padding='same', # padding을 same padding을 씀으로써 입력한 값과 같은 크기가 나온다.
    input_shape=(28,28,1) # cnn은 3차원을 인식하기 때문에 flatten을 써서 2차원을 1차원을 바꿔줄 필요가 없다.
))

# 풀링 층 만들기 : 지금은 maxPooling 사용할 거임
model.add(keras.layers.MaxPooling2D(
    2 # 요 값은 kernel_size로 준 값보다 작아야 한다?? 약간 애매 ?? 한 번 물어봐야 할듯
))

# 위에 한거 한 번 더 하기
# 이미지를 한번 더 축소 시키는 느낌??
model.add(keras.layers.Conv2D(
    64,
    kernel_size=3,
    activation='relu',
    padding='same'
))

model.add(keras.layers.MaxPooling2D(
    2
))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Dense 층 만들기
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10,activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 100)               3

In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('./Data/best-cnn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)

hist = model.fit(train_scaled,
                 train_target,
                 epochs=20,
                 validation_data = (val_scaled,val_target),
                 callbacks=[checkpoint_cb,early_stopping_cb]
                 )

Epoch 1/20


2023-03-02 14:08:10.089162: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1500/1500 [==============================] - 15s 10ms/step - loss: 0.4979 - accuracy: 0.8206 - val_loss: 0.3305 - val_accuracy: 0.8802
Epoch 2/20
1500/1500 [==============================] - 15s 10ms/step - loss: 0.3321 - accuracy: 0.8823 - val_loss: 0.2746 - val_accuracy: 0.8980
Epoch 3/20
1500/1500 [==============================] - ETA: 0s - loss: 0.2854 - accuracy: 0.8980

KeyboardInterrupt: 

In [ ]:
hist.history.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(
    hist.history['loss']
)

plt.plot(
    hist.history['val_loss']
)

plt.xlabel('epoch')
plt.ylabel('loss')

plt.show()

In [ ]:
plt.plot(
    hist.history['accuracy']
)

plt.plot(
    hist.history['val_accuracy']
)

plt.xlabel('epoch')
plt.ylabel('accuracy')

plt.show()

In [ ]:
# 검증데이터
model.evaluate(val_scaled, val_target)

In [ ]:
# 이미지 하나 골라서 잘 맞추나 보자.
# 이미지 보기
plt.imshow(
    val_scaled[0].reshape(28,28),
    cmap='gray_r'
)

In [ ]:
import numpy as np
np.argmax(model.predict(val_scaled[0:1]))

In [ ]:
# 0 - 9 글자로 바꾸기
classes = [
    't-shirt',
    'pants',
    'sweather',
    'dress',
    'coat',
    'sandle',
    'shirt',
    'sneakers',
    'bag',
    'angle boots'
]

In [ ]:
classes[np.argmax(model.predict(val_scaled[0:1]))]

### 시험 데이터로 확인하기
- train, validate 로 끝난 모델을 가지고 한번만 테스트 해보는 것이다.

In [ ]:
test_scaled = test_input.reshape(-1,28,28,1) / 255.0

In [ ]:
model.evaluate(test_scaled,test_target)

In [ ]:
plt.imshow(
    test_scaled[4].reshape(28,28),
    cmap='gray_r'
)

plt.show()

In [ ]:
classes[np.argmax(model.predict(test_scaled[4:5]))]